In [ ]:
# Intrusion Detection Analysis & Visualization
# Run this notebook AFTER executing main_pipeline.py
# 
# Fixed:
# - Proper file path handling
# - Empty data validation
# - Consistent styling
# - Accurate axis labels

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import numpy as np
from pathlib import Path

# Configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300

# File paths
OUTPUT_DIR = Path('../output/processed')
DATA_FILE = OUTPUT_DIR / 'processed_logs.csv'

print("="*70)
print("INTRUSION DETECTION - VISUAL ANALYSIS")
print("="*70)

# Validate file exists
if not DATA_FILE.exists():
    print(f"\n❌ Error: {DATA_FILE} not found!")
    print("Please run main_pipeline.py first to generate data.")
    raise FileNotFoundError(f"Data file not found: {DATA_FILE}")

# Load data
df = pd.read_csv(DATA_FILE)
df['timestamp'] = pd.to_datetime(df['timestamp'])

print(f"\n✓ Loaded {len(df):,} processed log entries")
print(f"  Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")
print(f"  Unique IPs: {df['source_ip'].nunique():,}")
print(f"  Unique users: {df['username'].nunique():,}")

# Validate data
if df.empty:
    print("\n❌ Error: DataFrame is empty!")
    raise ValueError("No data to visualize")

# ===============================================
# VISUALIZATION 1: Timeline Analysis
# ===============================================
print("\n📊 Creating timeline analysis...")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# 1a: Failed vs Successful over time
hourly_failures = df[df['is_failed_login']].set_index('timestamp').resample('1H').size()
hourly_success = df[~df['is_failed_login']].set_index('timestamp').resample('1H').size()

ax1.plot(hourly_failures.index, hourly_failures.values, 
         label='Failed Logins', color='#e74c3c', linewidth=2.5, marker='o', markersize=4)
ax1.plot(hourly_success.index, hourly_success.values, 
         label='Successful Logins', color='#27ae60', linewidth=2.5, marker='s', markersize=4)
ax1.set_title('Authentication Attempts Over Time (Hourly)', 
              fontsize=14, fontweight='bold', pad=20)
ax1.set_xlabel('Time', fontsize=11)
ax1.set_ylabel('Number of Attempts', fontsize=11)
ax1.legend(loc='upper left', fontsize=10)
ax1.grid(True, alpha=0.3, linestyle='--')
ax1.tick_params(axis='x', rotation=45)

# 1b: Top attacking IPs
top_attackers = df[df['is_failed_login']].groupby('source_ip').size().sort_values(ascending=True).tail(10)
colors_bar = ['#e74c3c' if val > 50 else '#e67e22' if val > 25 else '#f39c12' 
              for val in top_attackers.values]

ax2.barh(range(len(top_attackers)), top_attackers.values, color=colors_bar, edgecolor='black', linewidth=0.5)
ax2.set_yticks(range(len(top_attackers)))
ax2.set_yticklabels(top_attackers.index, fontsize=9)
ax2.set_title('Top 10 IPs by Failed Login Attempts', fontsize=14, fontweight='bold', pad=20)
ax2.set_xlabel('Number of Failed Attempts', fontsize=11)
ax2.set_ylabel('Source IP Address', fontsize=11)
ax2.grid(True, alpha=0.3, axis='x', linestyle='--')

# Add value labels on bars
for i, v in enumerate(top_attackers.values):
    ax2.text(v + 1, i, str(v), va='center', fontsize=9, fontweight='bold')

plt.tight_layout()
output_file_1 = OUTPUT_DIR / 'timeline_analysis.png'
plt.savefig(output_file_1, bbox_inches='tight')
plt.show()
print(f"  ✓ Saved: {output_file_1}")

# ===============================================
# VISUALIZATION 2: Multi-Panel Attack Analysis
# ===============================================
print("\n📊 Creating attack pattern heatmaps...")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 2a: Hour of Day Analysis
ax1 = axes[0, 0]
hour_pivot = df.groupby(['hour_of_day', 'is_failed_login']).size().unstack(fill_value=0)
# FIXED: Proper column labeling
hour_pivot.columns = ['Success', 'Failed']
sns.heatmap(hour_pivot.T, annot=True, fmt='d', cmap='RdYlGn_r', ax=ax1, 
            cbar_kws={'label': 'Number of Attempts'}, linewidths=0.5)
ax1.set_title('Login Attempts by Hour of Day', fontsize=14, fontweight='bold', pad=15)
ax1.set_xlabel('Hour of Day (24-hour format)', fontsize=11)
ax1.set_ylabel('Login Status', fontsize=11)

# 2b: Top Targeted Usernames
ax2 = axes[0, 1]
top_users = df[df['is_failed_login']].groupby('username').size().sort_values(ascending=False).head(10)
bars = ax2.bar(range(len(top_users)), top_users.values, color='#e74c3c', edgecolor='black', linewidth=0.8)
ax2.set_xticks(range(len(top_users)))
ax2.set_xticklabels(top_users.index, rotation=45, ha='right', fontsize=10)
ax2.set_title('Most Targeted Usernames', fontsize=14, fontweight='bold', pad=15)
ax2.set_xlabel('Username', fontsize=11)
ax2.set_ylabel('Failed Login Attempts', fontsize=11)
ax2.grid(True, alpha=0.3, axis='y', linestyle='--')

# Add value labels
for i, (bar, val) in enumerate(zip(bars, top_users.values)):
    ax2.text(i, val + max(top_users.values)*0.02, str(val), 
             ha='center', va='bottom', fontsize=9, fontweight='bold')

# 2c: Internal vs External Traffic
ax3 = axes[1, 0]
traffic_data = df.groupby(['is_internal_ip', 'is_failed_login']).size().unstack(fill_value=0)
traffic_data.index = ['External', 'Internal']
traffic_data.columns = ['Success', 'Failed']

traffic_data.plot(kind='bar', ax=ax3, color=['#27ae60', '#e74c3c'], 
                  width=0.7, edgecolor='black', linewidth=0.8)
ax3.set_title('Internal vs External Traffic Analysis', fontsize=14, fontweight='bold', pad=15)
ax3.set_xlabel('Traffic Source', fontsize=11)
ax3.set_ylabel('Number of Attempts', fontsize=11)
ax3.set_xticklabels(traffic_data.index, rotation=0, fontsize=11)
ax3.legend(['Successful', 'Failed'], loc='upper right', fontsize=10)
ax3.grid(True, alpha=0.3, axis='y', linestyle='--')

# 2d: Success Rate Comparison
ax4 = axes[1, 1]
success_rates = df.groupby('is_internal_ip')['is_failed_login'].apply(
    lambda x: (1 - x.mean()) * 100
)
success_rates.index = ['External IPs', 'Internal IPs']

bars = ax4.bar(range(len(success_rates)), success_rates.values, 
               color=['#e67e22', '#3498db'], width=0.6, edgecolor='black', linewidth=0.8)
ax4.set_xticks(range(len(success_rates)))
ax4.set_xticklabels(success_rates.index, fontsize=11)
ax4.set_title('Login Success Rate: Internal vs External', fontsize=14, fontweight='bold', pad=15)
ax4.set_ylabel('Success Rate (%)', fontsize=11)
ax4.set_ylim([0, 100])
ax4.axhline(y=50, color='red', linestyle='--', alpha=0.7, linewidth=2, label='50% Threshold')
ax4.grid(True, alpha=0.3, axis='y', linestyle='--')
ax4.legend(loc='lower right', fontsize=10)

# Add percentage labels
for i, (bar, val) in enumerate(zip(bars, success_rates.values)):
    ax4.text(i, val + 2, f'{val:.1f}%', ha='center', va='bottom', 
             fontsize=11, fontweight='bold')

plt.tight_layout()
output_file_2 = OUTPUT_DIR / 'attack_heatmap.png'
plt.savefig(output_file_2, bbox_inches='tight')
plt.show()
print(f"  ✓ Saved: {output_file_2}")

# ===============================================
# STATISTICAL SUMMARY
# ===============================================
print("\n" + "="*70)
print("📈 STATISTICAL SUMMARY")
print("="*70)

total_attempts = len(df)
failed_attempts = df['is_failed_login'].sum()
success_attempts = total_attempts - failed_attempts

print(f"\n🔢 Overall Statistics:")
print(f"  Total authentication attempts: {total_attempts:,}")
print(f"  Failed attempts: {failed_attempts:,} ({failed_attempts/total_attempts*100:.1f}%)")
print(f"  Successful attempts: {success_attempts:,} ({success_attempts/total_attempts*100:.1f}%)")

print(f"\n🌐 Network Statistics:")
print(f"  Unique source IPs: {df['source_ip'].nunique():,}")
print(f"  Unique usernames targeted: {df['username'].nunique():,}")
internal_count = df['is_internal_ip'].sum()
external_count = total_attempts - internal_count
print(f"  Internal traffic: {internal_count:,} ({internal_count/total_attempts*100:.1f}%)")
print(f"  External traffic: {external_count:,} ({external_count/total_attempts*100:.1f}%)")

print(f"\n⏰ Temporal Statistics:")
peak_hour = df.groupby('hour_of_day').size().idxmax()
peak_count = df.groupby('hour_of_day').size().max()
print(f"  Peak activity hour: {peak_hour}:00 ({peak_count:,} attempts)")

most_attacked = df[df['is_failed_login']].groupby('username').size()
if not most_attacked.empty:
    most_attacked_user = most_attacked.idxmax()
    most_attacked_count = most_attacked.max()
    print(f"  Most attacked username: '{most_attacked_user}' ({most_attacked_count:,} attempts)")

print(f"\n🚨 Security Alerts:")
external_failed = df[~df['is_internal_ip'] & df['is_failed_login']]
if len(external_failed) > 0:
    worst_external = external_failed.groupby('source_ip').size().idxmax()
    worst_count = external_failed.groupby('source_ip').size().max()
    print(f"  Worst offending IP: {worst_external} ({worst_count:,} failed attempts)")

print("\n" + "="*70)
print("✅ VISUALIZATION COMPLETE")
print(f"  Generated 2 visualization files in {OUTPUT_DIR}")
print("="*70 + "\n")